# Music Sentiment Analysis
by Ben Pfeffer

## Intro 
What emotions are expressed most commonly in the lyrics of popular music in the US? What emotions are less popular? Are different emotions correlated with one another? In this project I will analyze the emotional content of music lyrics in the popular music of the last 50 years. 

Keep in mind that I am only analyzing the emotional content of the *lyrics*, not the *melodies* in this project. Analyzing music while totally disregarding the melody is at best incomplete, so take everything you read here with a grain of salt. Sometimes the missing melody may be [amplifying](https://www.youtube.com/watch?v=4fWyzwo1xg0) the lyrical emotions explored here. Other times the tune [contrasts](https://www.youtube.com/watch?v=unfzfe8f9NI) with the lyrics, acting as an emotional counterpoint or overriding the lyrics altogether. There is lots of research being done in the interplay between [music and lyrics](https://www.researchgate.net/publication/247733383_Songs_and_emotions_Are_lyrics_and_melodies_equal_partners) that warrants further investigation. I think this project can provide valuable insight into what emotions Americans are explicitly indulging in through the musical lyrics, but even in the best case scenario this is only half the picture. 

This project was inspired by two data science projects I read about on the internet. The first is Greg Rafferty's [Sentiment Analysis on the Texts of Harry Potter](https://towardsdatascience.com/basic-nlp-on-the-texts-of-harry-potter-sentiment-analysis-1b474b13651d). The second is John W. Miller's [Trucks and Beer 🍺](https://www.johnwmillr.com/trucks-and-beer/). I decided to do a bit of a combination of the two, performing a similar sentiment analysis as Rafferty, but using Miller's Genius API wrapper to scrape lyrics from many musical artists. 

 The Billboard Hot 100 Artists is a year-end list of the most successful and prominent artists of the year. It measures record sales, radio airtime, and (for newer artists) internet streaming in the US. The lists for all years ranging between 1970 and 2019 are available online. For this reason I believe these lists will allow us to build a good sample of the most popular music in the US. One advantage to this approach is that it will allow us to get historical data from artists back before streaming was popular (Spotify wouldn't be able to tell us who people were listening to in the 70's). One limitation to this approach is that it only gives us data about the very top of the pyramid. There is still tons of music that is very popular in the US made by artists who have never made a Billboard Hot 100 Artists list. There is a risk that the emotional content of many of those artist's lyrics is significantly different than the emotional content of the artists at the top. 


 ## data wrangling 
 - describe how I got the data. 
 
 Links to github.
 

## methodology 
- how data was processed and analyzed. 

links to github. 

## Findings 
- more happy than sad, which emotions correlated, distributions for each emotion

## Conclusion 
- wrap up & suggestions for further investigation
    - scrape top 100 songs per year. emotional content over time analysis
    - take one artist and emotionally classify each album & compare (eg Beyonce)

poop on a shoe